In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/AI_Dataset/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt

In [0]:
from keras.models import load_model

GDRIVE_MODEL_PATH = "/content/gdrive/My Drive/AI_Dataset/resnetv2_model.h5"

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

# _cols_ = [col for col in one_hot_df.columns if not col.startswith("image_path")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        if self.augmentation is None:
          image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        else:
          image = np.stack([self.augmentation(image=cv2.imread(item["image_path"]))["image"] for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        # target = items[_cols_].values

        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=30, shuffle=False)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate, Normalize, MotionBlur, Cutout, MedianBlur, CenterCrop, Blur
)

# create train and validation data generators
training_augmentation = Compose([
    HorizontalFlip(p=0.5),
    RandomContrast(limit=0.5, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=0.5, p=0.5),
    CLAHE(p=1.0),
    #ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=0, p=0.8), 
    MotionBlur(blur_limit=7,p=0.6)
    # ToFloat(max_value=255),
    # Normalize()
])

#validation_augmentation = Compose([
    # CLAHE(p=1.0, clip_limit=2.0),
    #ToFloat(max_value=255)
#])

train_gen = PersonDataGenerator(train_df, batch_size=22, augmentation=training_augmentation)
valid_gen = PersonDataGenerator(val_df, batch_size=22, shuffle=False, augmentation=None)

cv2_imshow(train_gen[2][0][1])

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
# plot the total loss, category loss, and color loss
lossNames = ["gender_output_loss", "gender_output_acc",
             "image_quality_output_loss", "image_quality_output_acc",
             "age_output_loss", "age_output_acc", 
             "weight_output_loss", "weight_output_acc", 
             "bag_output_loss", "bag_output_acc", 
             "footwear_output_loss", "footwear_output_acc", 
             "pose_output_loss", "pose_output_acc", 
             "emotion_output_loss", "emotion_output_acc"]
def plot_model_losses(H, epochs):
  plt.style.use("ggplot")
  (fig, ax) = plt.subplots(8, 2, figsize=(20, 45))
  
  # loop over the loss names
  for (i, l) in enumerate(lossNames):
    r = int(i/2)
    c = i % 2
    # plot the loss for both the training and validation data
    title = format(l)
    ax[r][c].set_title(title)
    ax[r][c].set_xlabel("Epoch #")
    ax[r][c].set_ylabel("Loss" if c == 1 else "Accuracy")
    ax[r][c].plot(np.arange(0, epochs), H.history[l], label=l)
    ax[r][c].plot(np.arange(0, epochs), H.history["val_" + l],label="val_" + l)
    ax[r][c].set_xticks([])
    ax[r][c].legend()

  plt.show()


**ResNet V2 Definition**

In [0]:
from keras.regularizers import l2

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    x = Dropout(0.1)(x)
    return x

def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=10)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,kernel_initializer='he_normal')(y)
    #outputs = y

    # heads
    gender = build_head("gender", outputs)
    image_quality = build_head("image_quality", outputs)
    age = build_head("age", outputs)
    weight = build_head("weight", outputs)
    bag = build_head("bag", outputs)
    footwear = build_head("footwear", outputs)
    emotion = build_head("emotion", outputs)
    pose = build_head("pose", outputs)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion])
    # model = Model(inputs=inputs, outputs=outputs)
    return model

def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

**Model Initialization**

In [0]:
# !pip install --upgrade --upgrade-strategy only-if-needed https://github.com/faizanahemad/data-science-utils/tarball/master > /dev/null
# !pip install --upgrade --upgrade-strategy only-if-needed git+https://www.github.com/keras-team/keras-contrib.git > /dev/null

In [0]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
 return round(0.005 * 1/(1 + 0.319 * epoch), 10)

opt_sgd = SGD(lr=0.003, momentum=0.9)
opt_sgd_nesterov = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
opt_adam = Adam(lr=0.003)

losses = {
  "gender_output": "binary_crossentropy",
  "image_quality_output": "categorical_crossentropy",
  "age_output": "categorical_crossentropy",
  "weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy",

}
loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 20.0, "pose_output": 10.0, "emotion_output": 20.0}

In [0]:
model = resnet_v2(input_shape=(224,224,3), depth=20, num_classes=27)
model.compile(loss=losses,
              loss_weights=loss_weights,
              optimizer=opt_adam,
              metrics=['accuracy'])

In [0]:
# from data_science_utils.vision.keras import *

# lrf = LRFinder(model)
# model.metrics_names
#generator = datagen.flow(X_train, Y_train, batch_size=256,shuffle=True)
#test_generator = datagen_validation.flow(X_test, Y_test, batch_size=256, shuffle=True)
# lrf.find_generator(train_gen, 0.0001, 10.0, valid_gen, epochs=1, steps_per_epoch=None)
# lrf.plot_loss()

In [0]:
model.summary()

In [0]:
reduce_gender_lr = ReduceLROnPlateau(monitor='gender_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_img_quality_lr = ReduceLROnPlateau(monitor='image_quality_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_age_lr = ReduceLROnPlateau(monitor='age_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_weight_lr = ReduceLROnPlateau(monitor='weight_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_bag_lr = ReduceLROnPlateau(monitor='bag_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_footwear_lr = ReduceLROnPlateau(monitor='footwear_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_pose_lr = ReduceLROnPlateau(monitor='pose_output_loss', factor=0.2, patience=3, min_lr=0.001)
reduce_emotion_lr = ReduceLROnPlateau(monitor='emotion_output_loss', factor=0.2, patience=3, min_lr=0.001)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1,
    class_weight={0: 1., 1: 5., 2: 50., 3: 50., 4: 10., 5: 2., 6: 2., 7: 50.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler)]
)

In [0]:
keras.backend.eval(model.optimizer.lr)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=5,
    verbose=1,
    class_weight={0: 1., 1: 100., 2: 100., 3: 100., 4: 100., 5: 2., 6: 2., 7: 200.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr]
)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1,
    class_weight={0: 1., 1: 5., 2: 50., 3: 50., 4: 10., 5: 2., 6: 2., 7: 50.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler)]
)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1,
    class_weight={0: 1., 1: 5., 2: 50., 3: 50., 4: 10., 5: 2., 6: 2., 7: 200.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler)]
)

In [0]:
model = load_model(GDRIVE_MODEL_PATH)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=5,
    verbose=1,
    class_weight={0: 20., 1: 100., 2: 100., 3: 100., 4: 100., 5: 1., 6: 1., 7: 50.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler)]
)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1,
    class_weight={0: 20., 1: 100., 2: 100., 3: 100., 4: 100., 5: 1., 6: 1., 7: 50.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler)]
)

In [0]:
keras.backend.eval(model.optimizer.lr)

In [0]:
model = load_model(GDRIVE_MODEL_PATH)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1,
    class_weight={0: 1., 1: 100., 2: 100., 3: 50., 4: 50., 5: 1., 6: 1., 7: 100.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler)]
)

In [0]:
keras.backend.eval(model.optimizer.lr)

In [0]:
def scheduler(epoch, lr):
 return round(0.002 * 1/(1 + 0.319 * epoch), 10)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=20,
    initial_epoch=10,
    verbose=1,
    class_weight={0: 1., 1: 100., 2: 100., 3: 50., 4: 50., 5: 1., 6: 1., 7: 100.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler, verbose=1)]
)

In [0]:
model = load_model(GDRIVE_MODEL_PATH)

In [0]:
keras.backend.eval(model.optimizer.lr)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=30,
    initial_epoch=20,
    verbose=1,
    class_weight={0: 1., 1: 100., 2: 100., 3: 50., 4: 50., 5: 1., 6: 1., 7: 100.},
    callbacks=[reduce_gender_lr,reduce_img_quality_lr,reduce_age_lr,reduce_weight_lr,
               reduce_bag_lr,reduce_footwear_lr,reduce_pose_lr,reduce_emotion_lr,
               LearningRateScheduler(scheduler, verbose=1)]
)

In [0]:
model.save(GDRIVE_MODEL_PATH)
print("Saved model to disk")

In [0]:
def evaluate_model(model):
    results = model.evaluate_generator(valid_gen, verbose=1)
    accuracies = {}
    losses = {}
    for k, v in zip(model.metrics_names, results):
        if k.endswith('acc'):
            accuracies[k] = round(v * 100, 4) 
        else:
            losses[k] = v
    return accuracies

evaluate_model(model)